In [1]:
import os
import sys

os.chdir("C:/dataanalytics/python")
os.curdir

#Configure the environment . Set this up to the directory where spark is installed
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\\spark'
    
#create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exists. The names might change as
#versions change
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.6-src.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
 
#Initialize a spark context
from pyspark import SparkContext
from pyspark import SparkConf

#optionally configure spark
conf = (SparkConf().setAppName("sqldatatypes").setMaster("local[2]").set("spark.executor.memory", "1g"))

#Initalize spark context onl runs once
sc = SparkContext(conf=conf)

In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql import DataFrame 
from pyspark.sql import Column 
from pyspark.sql import Row 
from pyspark.sql import GroupedData
from pyspark.sql import DataFrameNaFunctions 
from pyspark.sql import DataFrameStatFunctions
from pyspark.sql import functions 
from pyspark.sql import types 
from pyspark.sql import Window 
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext

In [3]:
spark = SparkSession.builder.appName("sqldatatypes").master('local[2]').config(conf=conf).getOrCreate()

In [9]:
 lines = sc.textFile(r"C:\spark\examples\src\main\resources\people.txt")

In [10]:
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age = int(p[1])))

In [11]:
#create a datftrame and infer schema
schemaPeople = spark.createDataFrame(people)


In [12]:
schemaPeople.createOrReplaceTempView("people")

In [13]:
teenagers = spark.sql("select name from people where age >= 13 and age <= 19 ")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



In [14]:
# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: "Name:" + p.name).collect()
for name in teenNames:
    print(name)

Name:Justin


In [4]:
df = spark.read.load(r"C:\spark\examples\src\main\resources\users.parquet")

In [18]:
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [23]:
df.select("name", "favorite_numbers").write.save("C:\spark\examples\src\main\resources\namesandfavcolors.parquet")

Py4JJavaError: An error occurred while calling o335.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperatio

In [24]:
df.select("name", "favorite_color").write.save("namesAndFavColors.parquet")

AnalysisException: 'path file:/C:/dataanalytics/python/namesAndFavColors.parquet already exists.;'

In [7]:
df.write.mode("append")\
.parquet("C:\spark\examples\src\main\resources\namesandfavcolors.parquet")
df.write.mode("overwrite").format("parquet").save("C:\spark\examples\src\main\resources\namesandfavcolors.parquet")

Py4JJavaError: An error occurred while calling o116.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpera